In [34]:
import imp
# import mutils
from mutils import PIL_test

In [35]:
imp.reload(PIL_test)
PIL_test.test_cnn(4, 3, 16)

mutils.PIL_test
0.8.1
    0 img passed loss = 4.101e+03
odict_keys(['bn.weight', 'bn.bias', 'bn.running_mean', 'bn.running_var', 'bn.num_batches_tracked', 'conv1.0.weight', 'conv1.0.bias', 'fc.weight', 'fc.bias'])
bn.weight: std=0.0011547200847417116, mean=1.000333309173584, abs_max=1.0010000467300415
bn.bias: std=0.0011547004105523229, mean=0.00033333327155560255, abs_max=0.0009999999310821295
bn.running_mean: std=0.006404982414096594, mean=0.04238330200314522, abs_max=0.049718666821718216
bn.running_var: std=0.0046690599992871284, mean=0.8304567933082581, abs_max=0.8357672691345215
bn.num_batches_tracked: 

RuntimeError: std_mean only support floating-point dtypes